# Outlier statistics

In [1]:
from tqdm.notebook import tqdm
from utils.utils import *

In [2]:
# get all files from participants
folders = get_participant_folder_per_participant(raw_folder=False)
print(folders)
manual_evaluations = {participant:{} for participant in folders}
global FIXATION_CORRECTION_ITERATION 
FIXATION_CORRECTION_ITERATION[0] = 4
print('Fixation correction iteration:', current_fixation_correction_iteration())
print('Previous Fixation correction iteration:', previous_fixation_correction_iteration())
for iteration in range(1,current_fixation_correction_iteration()+1):
    manual_evaluation_paths:dict[str,Path] = exist_previous_manual_accuracy_evaluation_paths(lambda:iteration)
    for participant in folders:
        manual_evaluations[participant][iteration] = pd.read_csv(manual_evaluation_paths[participant], sep=SEPARATOR, index_col=False, dtype={PARTICIPANT: str})

in subfolder ../08-Data-Trial_Recordings/processed/total, no participant number could be identified.
{'003': WindowsPath('../08-Data-Trial_Recordings/processed/003'), '004': WindowsPath('../08-Data-Trial_Recordings/processed/004'), '005': WindowsPath('../08-Data-Trial_Recordings/processed/005'), '007': WindowsPath('../08-Data-Trial_Recordings/processed/007'), '008': WindowsPath('../08-Data-Trial_Recordings/processed/008'), '009': WindowsPath('../08-Data-Trial_Recordings/processed/009'), '010': WindowsPath('../08-Data-Trial_Recordings/processed/010'), '011': WindowsPath('../08-Data-Trial_Recordings/processed/011'), '012': WindowsPath('../08-Data-Trial_Recordings/processed/012'), '013': WindowsPath('../08-Data-Trial_Recordings/processed/013'), '014': WindowsPath('../08-Data-Trial_Recordings/processed/014'), '015': WindowsPath('../08-Data-Trial_Recordings/processed/015'), '016': WindowsPath('../08-Data-Trial_Recordings/processed/016'), '017': WindowsPath('../08-Data-Trial_Recordings/proce

### Calculate special fixations with multiple algorithms

In [3]:
outliers = {participant:{} for participant in folders}# all trials with outliers
all_outliers = {participant:{} for participant in folders}
non_excluded = 0
all_trials = 0
excluded_in_correction = {participant:set() for participant in folders}
for participant in tqdm(folders):
    for iteration, manual_evaluation in manual_evaluations[participant].items():
        participant_outliers = manual_evaluation[manual_evaluation[MANUAL_ACCURACY_EVALUATION_OUTLIER]>0]
        for i, row in participant_outliers.iterrows():
            snippet = row[SNIPPET]
            if not snippet in outliers[participant]:
                outliers[participant][snippet]=0
                all_outliers[participant][snippet]=0
            outliers[participant][snippet]+=row[MANUAL_ACCURACY_EVALUATION_OUTLIER]
            all_outliers[participant][snippet]+=row[MANUAL_ACCURACY_EVALUATION_OUTLIER]
        participant_exclusions = manual_evaluation[manual_evaluation[MANUAL_ACCURACY_EVALUATION_CHOICE]==FIXATION_CORRECTION_ALGORITHM_SNIPPET_EXCLUDED]
        excluded_in_correction[participant] |= set(participant_exclusions[SNIPPET].values)
    exclusions = get_exclusions(participant, [SNIPPET], [VISUAL, BEHAVIORAL])[SNIPPET]
    for snippet in exclusions:
        if any(exclusions[snippet].values()):
            if snippet in outliers[participant]:
                outliers[participant].pop(snippet)
            if exclusions[snippet][BEHAVIORAL]:
                if snippet in excluded_in_correction[participant]:
                    excluded_in_correction[participant].remove(snippet)
                pass
            elif exclusions[snippet][VISUAL][0][:4]<'01d7':
                if snippet in excluded_in_correction[participant]:
                    excluded_in_correction[participant].remove(snippet)
                pass
            else:
                all_trials+=1
        else:
            non_excluded+=1
            all_trials+=1

print('Outlier distribution across non-excluded trials:',outliers)
print('Total number of non-excluded trials containing outliers', sum([len(snippets) for snippets in outliers.values()]))
print('Relative number of non-excluded trials containing outliers', sum([len(snippets) for snippets in outliers.values()])/non_excluded, 
      'on basis of', non_excluded,'non-excluded trials')
print('Total number of outliers in non-excluded trials', sum([sum(snippets.values()) for snippets in outliers.values()]))



print('Outlier distribution across all trials:',all_outliers)
print('Total number of all trials containing outliers', sum([len(snippets) for snippets in all_outliers.values()]))
print('Relative number of all trials containing outliers', sum([len(snippets) for snippets in all_outliers.values()])/all_trials, 
      'on basis of all', all_trials,'trials')
print('Total number of outliers in all trials', sum([sum(snippets.values()) for snippets in all_outliers.values()]))

print('Excluded in correction distribution across all trials:',excluded_in_correction)
print('Total number of all trials excluded in correction', sum([len(snippets) for snippets in excluded_in_correction.values()]))
# for each trial, accumulate the number of outliers
# only those which are not excluded

  0%|          | 0/24 [00:00<?, ?it/s]

Outlier distribution across non-excluded trials: {'003': {'0-clean-v1': 2, '5-clean-v0': 2, '8-obf-v0': 1, '9-clean-v0': 1, '10-obf-v1': 1, '11-obf-v1': 1, '13-clean-v0': 2, '13-clean-v1': 1, '14-obf-v1': 1, '15-clean-v0': 1, '15-clean-v1': 1, '15-clean-v2': 1, '17-obf-v0': 1, '17-obf-v1': 1, '49-obf-v0': 1}, '004': {'0-obf-v1': 2, '8-clean-v0': 1, '12-obf-v1': 3, '14-clean-v2': 1, '30-obf-v1': 1, '54-clean-v0': 2, '54-clean-v2': 2, '62-clean-v0': 1}, '005': {'11-obf-v1': 1, '15-clean-v0': 2, '30-clean-v1': 1, '57-obf-v0': 1, '57-obf-v1': 2, '60-obf-v2': 2, '62-obf-v1': 1}, '007': {'10-obf-v0': 1, '13-obf-v0': 1, '16-obf-v0': 1, '49-clean-v1': 1, '60-clean-v1': 1, '16-obf-v1': 1}, '008': {'5-obf-v1': 2, '5-obf-v2': 1, '9-clean-v2': 1, '10-clean-v1': 1, '11-obf-v2': 1, '13-clean-v0': 2, '14-obf-v0': 1, '15-clean-v1': 1, '15-clean-v2': 1, '16-clean-v0': 1, '49-obf-v0': 1, '54-clean-v1': 3, '54-clean-v2': 1, '60-obf-v2': 1, '61-obf-v0': 1, '62-clean-v0': 2, '62-clean-v1': 1, '57-clean-v2'